In [4]:
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.offline as py
import numpy as np

# Dateiname der Trajektoriedaten
camera_data = '/home/bastian/uni/visual_odometry_playground/orb_slam3/orb_slam3_ws/CameraTrajectory.csv'
kf_data = '/home/bastian/uni/visual_odometry_playground/orb_slam3/orb_slam3_ws/KeyFrameTrajectory.csv'

# Listen für Delta-Werte
cam_delta_twb_x = []
cam_delta_twb_y = []
cam_delta_twb_z = []
kf_delta_twb_x = []
kf_delta_twb_y = []
kf_delta_twb_z = []

# Trajektoriedaten einlesen und Delta-Werte in den Listen speichern
with open(camera_data, 'r') as f:
    for line in f:
        teile = line.split()
        cam_delta_twb_x.append(float(teile[1]))
        cam_delta_twb_y.append(float(teile[2]))
        cam_delta_twb_z.append(float(teile[3]))
with open(kf_data, 'r') as f:
    for line in f:
        teile = line.split()
        kf_delta_twb_x.append(float(teile[1]))
        kf_delta_twb_y.append(float(teile[2]))
        kf_delta_twb_z.append(float(teile[3]))

# Rotationsmatrix für 45 Grad Drehung um die y-Achse
theta = np.radians(180)
rotation_matrix = np.array([[np.cos(theta), 0, np.sin(theta)],
                            [0, 1, 0],
                            [-np.sin(theta), 0, np.cos(theta)]])

# Daten um 45 Grad drehen
rot_cam_coords = np.dot(rotation_matrix, np.vstack([cam_delta_twb_x, cam_delta_twb_y, cam_delta_twb_z]))
rot_kf_coords = np.dot(rotation_matrix, np.vstack([kf_delta_twb_x, kf_delta_twb_y, kf_delta_twb_z]))

# Erstelle interaktiven 3D-Plot
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'scatter3d'}]])

# Plot hinzufügen
fig.add_trace(go.Scatter3d(x=rot_cam_coords[0], y=rot_cam_coords[1], z=rot_cam_coords[2], mode='lines', name='Camera'))
fig.add_trace(go.Scatter3d(x=rot_kf_coords[0], y=rot_kf_coords[1], z=rot_kf_coords[2], mode='markers', name='KeyFrames'))

# Layout anpassen
fig.update_layout(title='Getrackter Pfad',
                  scene=dict(xaxis=dict(title='Z-Koordinate'),  # Ändere die Beschriftung der Achsen
                             yaxis=dict(title='Y-Koordinate'),
                             zaxis=dict(title='X-Koordinate')),  # Ändere die Beschriftung der Achsen
                  margin=dict(l=0, r=0, t=30, b=0))

# Plot anzeigen
# py.plot(fig, filename='/home/bastian/uni/visual_odometry_playground/orb_slam3/orb_slam3_ws/getrackter_pfad.html')
py.iplot(fig)